In [1]:
import glob
import os
import gc

### Run all other models

In [2]:
files = glob.glob('data/*')

In [3]:
tasmax_prefix_size = len('data/tasmax_')
pr_prefix_size = len('data/pr_')
historical_suffix_size = len('_historical_Arg.nc')
future_suffix_size = len('_ssp585_Arg.nc')

In [4]:
model_names = [x[tasmax_prefix_size:-historical_suffix_size] for x in files if x[0:tasmax_prefix_size] == 'data/tasmax_' and x[-historical_suffix_size:] == '_historical_Arg.nc']

In [5]:
file_pairs = []

for suffix in ['_historical_Arg', '_ssp585_Arg']:
    for model in model_names:
        file_pairs.append({'pr': 'data/pr_'+model+suffix+'.nc',
                           'tas': 'data/tasmax_'+model+suffix+'.nc',
                           'results_path': 'results/'+model+suffix})

In [6]:
import psutil
process = psutil.Process()

for record in file_pairs:
    print(f"Writting results to {record['results_path']}")
    os.makedirs(record['results_path'], exist_ok=True)
    
    bash_prompt = f"python process_data.py --file_name_temp {record['tas']} --file_name_pp {record['pr']} --results_folder {record['results_path']}"
    os.system(bash_prompt)
    gc.collect()
    print(' ')
    print(f'Currently using {(process.memory_info().rss/1024/1024)}MB of memory')
    print(' ')    

Writting results to results/CNRM-CM6-1_ssp585_Arg
Loading temperature data from: data/tasmax_CNRM-CM6-1_ssp585_Arg.nc
Loading precipitation data from: data/pr_CNRM-CM6-1_ssp585_Arg.nc

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 0
Results saved to: results/CNRM-CM6-1_ssp585_Arg/map_winter_pp_1_strat_0_extremes_local.csv
Number of unique stations processed: 468
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 1
Results saved to: results/CNRM-CM6-1_ssp585_Arg/map_winter_pp_1_strat_1_extremes_local.csv
Number of unique stations processed: 468
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 2
Results saved to: results/CNRM-CM6-1_ssp585_Arg/map_winter_pp_1_strat_2_extremes_local.csv
Number of unique stations processed: 468
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 3
Results saved to: results/CNRM-CM6-1_ssp585_Arg/map_winter_pp_1_strat_3_extremes_l

Traceback (most recent call last):
  File "/Users/mcamporino/workspace/fonsa/project_lag_paper9/current_code/process_data.py", line 178, in <module>
    main()
  File "/Users/mcamporino/workspace/fonsa/project_lag_paper9/current_code/process_data.py", line 83, in main
    df_raw['time'] = pd.to_datetime(df_raw.time.astype(str))
  File "/Users/mcamporino/.pyenv/versions/3.10.5/lib/python3.10/site-packages/pandas/core/tools/datetimes.py", line 1063, in to_datetime
    cache_array = _maybe_cache(arg, format, cache, convert_listlike)
  File "/Users/mcamporino/.pyenv/versions/3.10.5/lib/python3.10/site-packages/pandas/core/tools/datetimes.py", line 247, in _maybe_cache
    cache_dates = convert_listlike(unique_dates, format)
  File "/Users/mcamporino/.pyenv/versions/3.10.5/lib/python3.10/site-packages/pandas/core/tools/datetimes.py", line 433, in _convert_listlike_datetimes
    return _array_strptime_with_fallback(arg, name, utc, format, exact, errors)
  File "/Users/mcamporino/.pyenv/versi

 
Currently using 50.703125MB of memory
 
Writting results to results/TaiESM1_ssp585_Arg
Loading temperature data from: data/tasmax_TaiESM1_ssp585_Arg.nc
Loading precipitation data from: data/pr_TaiESM1_ssp585_Arg.nc

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 0
Results saved to: results/TaiESM1_ssp585_Arg/map_winter_pp_1_strat_0_extremes_local.csv
Number of unique stations processed: 722
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 1
Results saved to: results/TaiESM1_ssp585_Arg/map_winter_pp_1_strat_1_extremes_local.csv
Number of unique stations processed: 722
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 2
Results saved to: results/TaiESM1_ssp585_Arg/map_winter_pp_1_strat_2_extremes_local.csv
Number of unique stations processed: 722
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 3
Results saved to: results/TaiESM1_ssp585_Arg/map_winter_pp

### Run ERA

In [7]:
from create_era_csv import generate_csv_era_data

In [8]:
import datetime

print(datetime.datetime.now())
generate_csv_era_data(raw_data_path='era_data', output_path='era')

2025-09-03 22:53:02.785137
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


In [1]:
# Run results for ERA
import netCDF4 as nc
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from global_land_mask import globe
# Assuming aux_ncdf.py is in the same directory or accessible via PYTHONPATH
from aux_ncdf import preprocess_data, add_max_increasing_chain, add_max_high_temp_chain, add_large_jump, calculate_increasing_temp_features, calculate_high_temp_features, calculate_temp_jump_features
import argparse
import os

df = pd.read_csv('era/daily_era_data.csv')

In [2]:
df.shape

(229822100, 8)

In [3]:
df.latitude.min()

-56.0

In [4]:
df.latitude.max()

-20.0

In [5]:
df.longitude.min()

-76.0

In [6]:
df.longitude.max()

-52.0

In [7]:
df = df[df.latitude < -38] # longitude >= -64

In [8]:
df.shape

(114118560, 8)

In [9]:
results_folder = 'results/era_historical_Arg'

In [10]:
# Loop through different processing scenarios
for season in ['winter', 'summer']:
    for pp_lower_bound in [1]:  # [1, 5]
        for grouping_strategy in [0, 1, 2, 3]:
            print(f'\n--- Processing Scenario ---')
            print(f'Season: {season}')
            print(f'PP lower bound: {pp_lower_bound}')
            print(f'Grouping strategy: {grouping_strategy}')

            extreme_value_strategy = 'local'

            # Preprocess data based on current scenario
            df_prep = preprocess_data(
                df,
                season=season,
                pp_lower_bound=pp_lower_bound,
                grouping_strategy=grouping_strategy,
                extreme_value_strategy=extreme_value_strategy
            )

            # Add various chain and jump features
            df_prep = add_max_increasing_chain(df_prep, eps=0.5)
            df_prep = add_max_high_temp_chain(df_prep, perc=0.75)
            df_prep = add_large_jump(df_prep, perc=0.9, rain_delay=1)

            # Calculate extreme precipitation rates and values
            extreme_pp_rate = df_prep.groupby('station').PP_extreme.agg(['mean', 'count']).reset_index()
            extreme_pp_rate.columns = ['station', 'extreme_pp_rate', 'days_total']
            
            days_with_rain = df_prep.groupby('station').rain.sum().to_frame('days_with_pp').reset_index()
            extreme_pp_rate = pd.merge(extreme_pp_rate, days_with_rain, on='station', how='left')

            # Note: PP.min() here seems to imply the minimum PP value when PP_extreme is 1.
            # If it's meant to be the *maximum* extreme PP value, you might want .max()
            extreme_pp_value = df_prep[df_prep.PP_extreme == 1].groupby('station').PP.min().reset_index().rename(columns={'PP': 'extreme_pp_value'})
            extreme_pp_rate = pd.merge(extreme_pp_rate, extreme_pp_value, on='station', how='left')

            # Calculate temperature features
            increasing_chain_stats = calculate_increasing_temp_features(df_prep)
            high_t_chain_stats = calculate_high_temp_features(df_prep)
            freq_large_jump_pos, pp_extreme_large_jump_pos, freq_large_jump_neg, pp_extreme_large_jump_neg = calculate_temp_jump_features(df_prep)

            pp_extreme_large_jump_pos = pp_extreme_large_jump_pos.rename(columns={'PP_extreme': 'large_jump_pos_extreme_pp_rate'})
            pp_extreme_large_jump_neg = pp_extreme_large_jump_neg.rename(columns={'PP_extreme': 'large_jump_neg_extreme_pp_rate'})

            # Merge all calculated features into a single results DataFrame
            results = pd.merge(extreme_pp_rate, increasing_chain_stats, on='station', how='left')
            results = pd.merge(results, high_t_chain_stats, on='station', how='left')
            results = pd.merge(results, freq_large_jump_pos, on='station', how='left')
            results = pd.merge(results, pp_extreme_large_jump_pos, on='station', how='left')
            results = pd.merge(results, freq_large_jump_neg, on='station', how='left')
            results = pd.merge(results, pp_extreme_large_jump_neg, on='station', how='left')

            # Add latitude, longitude, and land mask information
            results['lat'] = results.station.apply(lambda x: float(x.split('_')[0]))
            results['lon'] = results.station.apply(lambda x: float(x.split('_')[1]))
            results['is_land'] = results[['lat', 'lon']].apply(lambda r: globe.is_land(lat=r['lat'], lon=r['lon']), axis=1)

            # Calculate normalized extreme precipitation metrics
            extreme_pp_metrics = [
                'large_jump_pos_extreme_pp_rate',
                'large_jump_neg_extreme_pp_rate'
            ]

            for metric in extreme_pp_metrics:
                results[f'{metric}_norm'] = results[metric] / results.extreme_pp_rate

            # Save results to CSV
            output_filename = f'{results_folder}/map_{season}_pp_{pp_lower_bound}_strat_{grouping_strategy}_extremes_local__1.csv'
            results.to_csv(output_filename, index=False)
            print(f"Results saved to: {output_filename}")
            print(f"Number of unique stations processed: {results.station.nunique()}")
            print(' ')


--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 0
Results saved to: results/era_historical_Arg/map_winter_pp_1_strat_0_extremes_local__1.csv
Number of unique stations processed: 6984
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 1
Results saved to: results/era_historical_Arg/map_winter_pp_1_strat_1_extremes_local__1.csv
Number of unique stations processed: 6984
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 2
Results saved to: results/era_historical_Arg/map_winter_pp_1_strat_2_extremes_local__1.csv
Number of unique stations processed: 6984
 

--- Processing Scenario ---
Season: winter
PP lower bound: 1
Grouping strategy: 3
Results saved to: results/era_historical_Arg/map_winter_pp_1_strat_3_extremes_local__1.csv
Number of unique stations processed: 6984
 

--- Processing Scenario ---
Season: summer
PP lower bound: 1
Grouping strategy: 0
Results saved to: results/era_historical_Arg/

In [12]:
# Now merge files

results_folder = 'results/era_historical_Arg'

for season in ['winter', 'summer']:
    for pp_lower_bound in [1]:  # [1, 5]
        for grouping_strategy in [0, 1, 2, 3]:

            output_filename_core = f'{results_folder}/map_{season}_pp_{pp_lower_bound}_strat_{grouping_strategy}_extremes_local'
                        
            part1 = pd.read_csv(output_filename_core + '__0.csv')
            print(part1.shape)
            part2 = pd.read_csv(output_filename_core + '__1.csv')
            print(part2.shape)
            full = pd.concat([part1, part2], axis=0)
            print(full.shape)
            full.to_csv(output_filename_core + '.csv', index=False)
            

(7081, 20)
(6984, 20)
(14065, 20)
(7081, 20)
(6984, 20)
(14065, 20)
(7081, 20)
(6984, 20)
(14065, 20)
(7081, 20)
(6984, 20)
(14065, 20)
(7081, 20)
(6984, 20)
(14065, 20)
(7081, 20)
(6984, 20)
(14065, 20)
(7081, 20)
(6984, 20)
(14065, 20)
(7081, 20)
(6984, 20)
(14065, 20)
